In [12]:
import requests
import json
import pandas as pd
import numpy as np
from dataclasses import dataclass
from datetime import datetime
from typing import List, Optional

In [13]:
@dataclass
class F1Session:
    """Class representing an F1 session"""
    meeting_key: int
    session_key: int
    location: str
    date_start: str
    date_end: str
    session_type: str
    session_name: str
    country_key: int
    country_code: str
    country_name: str
    circuit_key: int
    circuit_short_name: str
    gmt_offset: str
    year: int
    
    def get_start_datetime(self) -> datetime:
        return datetime.fromisoformat(self.date_start.replace('Z', '+00:00'))
    
    def get_end_datetime(self) -> datetime:
        return datetime.fromisoformat(self.date_end.replace('Z', '+00:00'))
    
    def __str__(self) -> str:
        return f"{self.session_name} - {self.location} ({self.session_type}) - {self.date_start}"

@dataclass
class F1Driver:
    """Class representing an F1 driver"""
    meeting_key: int
    session_key: int
    driver_number: int
    broadcast_name: str
    full_name: str
    name_acronym: str
    team_name: str
    team_colour: str
    first_name: str
    last_name: str
    headshot_url: str
    country_code: Optional[str]
    
    def get_team_colour_hex(self) -> str:
        """Return team colour as a proper hex color code"""
        return f"#{self.team_colour}"
    
    def get_display_name(self) -> str:
        """Return a formatted display name"""
        return f"{self.first_name} {self.last_name}"
    
    def __str__(self) -> str:
        return f"#{self.driver_number} {self.full_name} ({self.name_acronym}) - {self.team_name}"


@dataclass
class F1Lap:
    """Class representing an F1 lap"""
    meeting_key: int
    session_key: int
    driver_number: int
    lap_number: int
    date_start: Optional[str]
    duration_sector_1: Optional[float]
    duration_sector_2: Optional[float]
    duration_sector_3: Optional[float]
    i1_speed: Optional[int]
    i2_speed: Optional[int]
    is_pit_out_lap: bool
    lap_duration: Optional[float]
    segments_sector_1: List[Optional[int]]
    segments_sector_2: List[Optional[int]]
    segments_sector_3: List[Optional[int]]
    st_speed: Optional[int]
    
    def get_start_datetime(self) -> Optional[datetime]:
        """Return the lap start time as a datetime object"""
        if self.date_start:
            return datetime.fromisoformat(self.date_start.replace('Z', '+00:00'))
        return None
    
    def get_total_lap_time(self) -> Optional[float]:
        """Return the total lap duration in seconds"""
        return self.lap_duration
    
    def get_sector_times(self) -> tuple[Optional[float], Optional[float], Optional[float]]:
        """Return sector times as a tuple (sector_1, sector_2, sector_3)"""
        return (self.duration_sector_1, self.duration_sector_2, self.duration_sector_3)
    
    def get_speeds(self) -> tuple[Optional[int], Optional[int], Optional[int]]:
        """Return speeds as a tuple (i1_speed, i2_speed, st_speed)"""
        return (self.i1_speed, self.i2_speed, self.st_speed)
    
    def is_complete_lap(self) -> bool:
        """Check if this is a complete lap with all sector times"""
        return all(time is not None for time in [self.duration_sector_1, self.duration_sector_2, self.duration_sector_3])
    
    def get_average_speed(self) -> Optional[float]:
        """Calculate average speed from available speed measurements"""
        speeds = [speed for speed in [self.i1_speed, self.i2_speed, self.st_speed] if speed is not None]
        return sum(speeds) / len(speeds) if speeds else None
    
    def __str__(self) -> str:
        lap_time_str = f"{self.lap_duration:.3f}s" if self.lap_duration else "N/A"
        return f"Lap #{self.lap_number} - Driver #{self.driver_number} - {lap_time_str}"
   

def get_sessions(year: int = 2025, session_type: str = None, country_name: str = None) -> Optional[List[F1Session]]:
    """
    Fetch F1 sessions for a given year and return as structured data objects
    
    Args:
        year: The year to fetch sessions for (default: 2025)
        
    Returns:
        List of F1Session objects or None if request fails
    """
    link = f"https://api.openf1.org/v1/sessions?"

    adds = f"year={year}"
    if session_type:
        adds += f"&session_type={session_type}"
    if country_name:
        adds += f"&country_name={country_name}"

    res = requests.get(link+adds)
    
    sessions = None
    if res.status_code == 200:
        json_data = res.json()
        sessions = [F1Session(**session_data) for session_data in json_data]
        print(f"Retrieved {len(sessions)} sessions for {year}")
        
    else:
        print(f"Failed to fetch sessions. Status code: {res.status_code}")
    
    return sessions

In [24]:
sessions = get_sessions(year=2025, session_type='Race')

if sessions:
    print(f"\nTotal sessions: {len(sessions)}")
    
    practice_sessions = [s for s in sessions if s.session_type == 'Practice']
    print(f"Practice sessions: {len(practice_sessions)}")

    qualifying_sessions = [s for s in sessions if s.session_type == 'Qualifying' and s.session_name == 'Qualifying']
    print(f"Qualifying sessions: {len(qualifying_sessions)}")

    race_sessions = [s for s in sessions if s.session_type == 'Race' and s.session_name == 'Race']
    print(f"Race sessions: {len(race_sessions)}")

    sprint_qualifying_sessions = [s for s in sessions if s.session_type == 'Qualifying' and s.session_name == 'Sprint Qualifying']
    print(f"Sprint qualifying sessions: {len(sprint_qualifying_sessions)}")

    sprint_sessions = [s for s in sessions if s.session_type == 'Race' and s.session_name == 'Sprint']
    print(f"Sprint Race sessions: {len(sprint_sessions)}")
    
    bahrain_sessions = [s for s in sessions if s.country_name == 'Bahrain']
    print(f"Bahrain sessions: {len(bahrain_sessions)}")
    
    if sessions:
        first_session = sessions[0]
        print(f"\nFirst session details:")
        print(f"  Location: {first_session.location}")
        print(f"  Session Type: {first_session.session_type}")
        print(f"  Start Time: {first_session.get_start_datetime()}")
        print(f"  End Time: {first_session.get_end_datetime()}")
        print(f"  Circuit: {first_session.circuit_short_name}")


Retrieved 19 sessions for 2025

Total sessions: 19
Practice sessions: 0
Qualifying sessions: 0
Race sessions: 16
Sprint qualifying sessions: 0
Sprint Race sessions: 3
Bahrain sessions: 1

First session details:
  Location: Melbourne
  Session Type: Race
  Start Time: 2025-03-16 04:00:00+00:00
  End Time: 2025-03-16 06:00:00+00:00
  Circuit: Melbourne


In [29]:
get_sessions(session_key=9908)

TypeError: get_sessions() got an unexpected keyword argument 'session_key'

In [30]:
sessions = get_sessions(year=2025, session_type='Qualifying')
sessions

Retrieved 19 sessions for 2025


[F1Session(meeting_key=1254, session_key=9689, location='Melbourne', date_start='2025-03-15T05:00:00+00:00', date_end='2025-03-15T06:00:00+00:00', session_type='Qualifying', session_name='Qualifying', country_key=5, country_code='AUS', country_name='Australia', circuit_key=10, circuit_short_name='Melbourne', gmt_offset='11:00:00', year=2025),
 F1Session(meeting_key=1255, session_key=9989, location='Shanghai', date_start='2025-03-21T07:30:00+00:00', date_end='2025-03-21T08:14:00+00:00', session_type='Qualifying', session_name='Sprint Qualifying', country_key=53, country_code='CHN', country_name='China', circuit_key=49, circuit_short_name='Shanghai', gmt_offset='08:00:00', year=2025),
 F1Session(meeting_key=1255, session_key=9994, location='Shanghai', date_start='2025-03-22T07:00:00+00:00', date_end='2025-03-22T08:00:00+00:00', session_type='Qualifying', session_name='Qualifying', country_key=53, country_code='CHN', country_name='China', circuit_key=49, circuit_short_name='Shanghai', gmt

In [16]:
def get_drivers(session_key: str = 'latest') -> Optional[List[F1Driver]]:
    """
    Fetch F1 Drivers for a given session key and return as structured data objects
    
    Args:
        session_key: The session key of the race or session (default: 'latest')
        
    Returns:
        List of F1Driver objects or None if request fails
    """
    link = f"https://api.openf1.org/v1/drivers?session_key={session_key}"

    res = requests.get(link)
    
    drivers = None
    if res.status_code == 200:
        json_data = res.json()
        # Convert JSON data to F1Driver objects
        drivers = [F1Driver(**driver_data) for driver_data in json_data]
        print(f"Retrieved {len(drivers)} drivers for session {session_key}")
        for driver in drivers[:3]:  # Print first 3 drivers
            print(f"  - {driver}")
        if len(drivers) > 3:
            print(f"  ... and {len(drivers) - 3} more drivers")
    else:
        print(f"Failed to fetch drivers. Status code: {res.status_code}")
    
    return drivers

In [17]:
# Example usage of the new F1Driver data type
drivers = get_drivers()

if drivers:
    print(f"\nTotal drivers: {len(drivers)}")
    
    # Example: Get all Red Bull Racing drivers
    red_bull_drivers = [d for d in drivers if d.team_name == 'Red Bull Racing']
    print(f"Red Bull Racing drivers: {len(red_bull_drivers)}")
    for driver in red_bull_drivers:
        print(f"  - {driver}")
    
    # Example: Get drivers by number range
    top_ten_drivers = [d for d in drivers if d.driver_number <= 10]
    print(f"\nDrivers with numbers 1-10: {len(top_ten_drivers)}")
    
    # Example: Access individual driver properties
    if drivers:
        first_driver = drivers[0]
        print(f"\nFirst driver details:")
        print(f"  Name: {first_driver.get_display_name()}")
        print(f"  Number: #{first_driver.driver_number}")
        print(f"  Team: {first_driver.team_name}")
        print(f"  Team Color: {first_driver.get_team_colour_hex()}")
        print(f"  Acronym: {first_driver.name_acronym}")
        print(f"  Headshot URL: {first_driver.headshot_url}")


Retrieved 20 drivers for session latest
  - #1 Max VERSTAPPEN (VER) - Red Bull Racing
  - #4 Lando NORRIS (NOR) - McLaren
  - #5 Gabriel BORTOLETO (BOR) - Kick Sauber
  ... and 17 more drivers

Total drivers: 20
Red Bull Racing drivers: 2
  - #1 Max VERSTAPPEN (VER) - Red Bull Racing
  - #22 Yuki TSUNODA (TSU) - Red Bull Racing

Drivers with numbers 1-10: 5

First driver details:
  Name: Max Verstappen
  Number: #1
  Team: Red Bull Racing
  Team Color: #4781D7
  Acronym: VER
  Headshot URL: https://media.formula1.com/d_driver_fallback_image.png/content/dam/fom-website/drivers/M/MAXVER01_Max_Verstappen/maxver01.png.transform/1col/image.png


In [18]:
# drivers

In [19]:
def get_laps(session_key: str = 'latest', driver_number: int = None) -> Optional[List[F1Lap]]:
    """
    Fetch F1 laps for a given session key and return as structured data objects
    
    Args:
        session_key: The session key of the race or session (default: 'latest')
        driver_number: Optional driver number to filter laps (default: None for all drivers)
        
    Returns:
        List of F1Lap objects or None if request fails
    """
    link = f"https://api.openf1.org/v1/laps?session_key={session_key}"
    
    if driver_number:
        link += f"&driver_number={driver_number}"

    res = requests.get(link)
    
    laps = None
    if res.status_code == 200:
        json_data = res.json()
        # Convert JSON data to F1Lap objects
        laps = [F1Lap(**lap_data) for lap_data in json_data]
        print(f"Retrieved {len(laps)} laps for session {session_key}")
        for lap in laps[:3]:  # Print first 3 laps
            print(f"  - {lap}")
        if len(laps) > 3:
            print(f"  ... and {len(laps) - 3} more laps")
    else:
        print(f"Failed to fetch laps. Status code: {res.status_code}")
    
    return laps

In [22]:
laps = get_laps()

Retrieved 976 laps for session latest
  - Lap #1 - Driver #27 - N/A
  - Lap #1 - Driver #1 - N/A
  - Lap #1 - Driver #4 - N/A
  ... and 973 more laps


In [23]:
laps

[F1Lap(meeting_key=1268, session_key=9912, driver_number=27, lap_number=1, date_start=None, duration_sector_1=None, duration_sector_2=None, duration_sector_3=None, i1_speed=None, i2_speed=None, is_pit_out_lap=False, lap_duration=None, segments_sector_1=[2048, 0, 0, 0, 0, 0], segments_sector_2=[0, 0, 0, 0, 0, 0, 0], segments_sector_3=[0, 0, 0, 0, 0, 0, 0, 0, 0], st_speed=None),
 F1Lap(meeting_key=1268, session_key=9912, driver_number=1, lap_number=1, date_start=None, duration_sector_1=None, duration_sector_2=29.088, duration_sector_3=28.517, i1_speed=321, i2_speed=309, is_pit_out_lap=False, lap_duration=None, segments_sector_1=[2048, 2049, 2049, 2049, 2049, 2049], segments_sector_2=[2051, 2049, 2049, 2051, 2051, 2049, 2049], segments_sector_3=[2051, 2049, 2049, 2049, 2049, 2049, 2049, 2049, 2051], st_speed=284),
 F1Lap(meeting_key=1268, session_key=9912, driver_number=4, lap_number=1, date_start=None, duration_sector_1=None, duration_sector_2=29.911, duration_sector_3=27.903, i1_speed=3

In [21]:
# Example usage of the new F1Lap data type
laps = get_laps(driver_number=63)  # Get laps for driver #63

if laps:
    print(f"\nTotal laps: {len(laps)}")
    
    # Example: Get complete laps only
    complete_laps = [lap for lap in laps if lap.is_complete_lap()]
    print(f"Complete laps: {len(complete_laps)}")
    
    # Example: Get fastest lap
    fastest_lap = min([lap for lap in laps if lap.lap_duration], key=lambda x: x.lap_duration)
    print(f"Fastest lap: {fastest_lap}")
    
    # Example: Analyze sector times
    if complete_laps:
        first_complete_lap = complete_laps[0]
        sector_times = first_complete_lap.get_sector_times()
        speeds = first_complete_lap.get_speeds()
        avg_speed = first_complete_lap.get_average_speed()
        
        print(f"\nFirst complete lap analysis:")
        print(f"  Lap time: {first_complete_lap.get_total_lap_time():.3f}s")
        print(f"  Sector 1: {sector_times[0]:.3f}s")
        print(f"  Sector 2: {sector_times[1]:.3f}s") 
        print(f"  Sector 3: {sector_times[2]:.3f}s")
        print(f"  I1 Speed: {speeds[0]} km/h")
        print(f"  I2 Speed: {speeds[1]} km/h")
        print(f"  ST Speed: {speeds[2]} km/h")
        print(f"  Average Speed: {avg_speed:.1f} km/h" if avg_speed else "  Average Speed: N/A")
    
    # Example: Get lap start times
    laps_with_times = [lap for lap in laps if lap.date_start]
    if laps_with_times:
        print(f"\nLaps with start times: {len(laps_with_times)}")
        for lap in laps_with_times[:3]:
            start_time = lap.get_start_datetime()
            print(f"  Lap {lap.lap_number}: {start_time}")


Retrieved 53 laps for session latest
  - Lap #1 - Driver #63 - N/A
  - Lap #2 - Driver #63 - 85.309s
  - Lap #3 - Driver #63 - 85.346s
  ... and 50 more laps

Total laps: 53
Complete laps: 52
Fastest lap: Lap #45 - Driver #63 - 81.800s

First complete lap analysis:
  Lap time: 85.309s
  Sector 1: 27.677s
  Sector 2: 29.339s
  Sector 3: 28.293s
  I1 Speed: 321 km/h
  I2 Speed: 334 km/h
  ST Speed: 338 km/h
  Average Speed: 331.0 km/h

Laps with start times: 52
  Lap 2: 2025-09-07 13:05:04.066000+00:00
  Lap 3: 2025-09-07 13:06:29.443000+00:00
  Lap 4: 2025-09-07 13:07:54.757000+00:00
